# High Resolution inference 

In [1]:
# run in the root of the repository
%load_ext autoreload
%autoreload 2
 
%cd ../..

/private/home/pfz/09-videoseal/videoseal-dev


/private/home/pfz/miniconda3/envs/img/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from videoseal.utils.display import save_vid
from videoseal.utils import Timer
from videoseal.evals.full import setup_model_from_checkpoint
from videoseal.evals.metrics import bit_accuracy, pvalue, capacity, psnr, ssim, msssim, linf
from videoseal.data.datasets import VideoDataset
from videoseal.augmentation import Identity, H264, Crop
from videoseal.modules.jnd import JND, JNDSimplified

import os
from tqdm import tqdm
import torch
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu" 
# device = "cuda" 

/private/home/pfz/miniconda3/envs/img/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os
from tqdm import tqdm
import torch

# Directory containing videos
video_dir = "assets/videos/"
base_output_folder = "outputs"
if not os.path.exists(base_output_folder):
    os.makedirs(base_output_folder)

# Example usage
ckpts = {
    # "128b": "/checkpoint/pfz/2025_logs/0306_vseal_ydisc_release/_nbits=128/checkpoint600.pth",
    "y_128b_img": "videoseal_1.0"
}

fps = 24 // 1
num_videos = None

# a timer to measure the time
timer = Timer()

# Iterate over all checkpoints
for model_name, ckpt in ckpts.items():
    wam = setup_model_from_checkpoint(ckpt)
    wam.eval()
    wam.compile()
    wam.to(device)

    # wam.blender.scaling_w = 0.2
    # wam.chunk_size = 200
    wam.step_size = 4
    wam.video_mode = "repeat"

    # Iterate over all video files in the directory
    video_files = [f for f in os.listdir(video_dir) if f.endswith(".mp4")][:num_videos]

    for video_file in tqdm(video_files, desc=f"Processing Videos for {model_name}"):
        video_path = os.path.join(video_dir, video_file)
        base_name = os.path.splitext(video_file)[0]

        # Load video (assuming a function `load_video` exists)
        timer.start()
        vid, mask = VideoDataset.load_full_video_decord(video_path)
        print(f"loading video {video_path} - took {timer.stop():.2f}s")

        # Watermark embedding
        timer.start()
        outputs = wam.embed(vid, is_video=True, lowres_attenuation=True)
        print(f"embedding watermark  - took {timer.stop():.2f}s")

        # compute diff
        imgs = vid  # b c h w
        imgs_w = outputs["imgs_w"]  # b c h w
        msgs = outputs["msgs"]  # b k
        diff = imgs_w - imgs

        # # save
        timer.start()
        save_vid(imgs, f"{base_output_folder}/{model_name}_{base_name}_ori.mp4", fps)
        save_vid(imgs_w, f"{base_output_folder}/{model_name}_{base_name}_wm.mp4", fps)
        save_vid(10*diff.abs(), f"{base_output_folder}/{model_name}_{base_name}_diff.mp4", fps)

        # Metrics
        metrics = {
            "psnr": psnr(imgs, imgs_w, is_video=True).mean().item(),
            # "ssim": ssim(imgs, imgs_w).mean().item(),
            # "msssim": msssim(imgs, imgs_w).mean().item(),
            # "linf": linf(imgs, imgs_w).mean().item()
        }

        # Augment video
        print(f"compressing and detecting watermarks")
        for ii in range(4):
        # for ii in range(1):
            if ii == 0:
                imgs_aug = imgs_w
                label = "Original"
            if ii == 1: 
                imgs_aug, _ = H264()(imgs_w, crf=30)
                imgs_aug, _ = Crop()(imgs_aug, size=0.75)
                label = "H264 30 + Crop 0.8"
            if ii == 2: 
                imgs_aug, _ = H264()(imgs_w, crf=40)
                label = "H264 40"
            if ii == 3: 
                imgs_aug, _ = H264()(imgs_w, crf=50)
                label = "H264 50"

            # detect
            timer.start()
            aggregate = True
            if not aggregate:
                outputs = wam.detect(imgs_aug, is_video=True)
                preds = outputs["preds"]
                bit_preds = preds[:, 1:]  # b k ...
                bit_accuracy_ = bit_accuracy(
                    bit_preds,
                    msgs
                ).nanmean().item()
                metrics[f"bit_accuracy_{label.lower().replace(' ', '_').replace('.', '')}"] = bit_accuracy_
                # print(f"{label} - Bit Accuracy: {bit_accuracy_:.3f} - took {timer.stop():.2f}s")
            else:
                bit_preds = wam.detect_and_aggregate(imgs_aug)
                bit_accuracy_ = bit_accuracy(
                    bit_preds,
                    msgs[:1]
                ).nanmean().item()
                pvalue_ = pvalue(
                    bit_preds,
                    msgs[:1]
                ).nanmean().item()
                capacity_ = capacity(
                    bit_preds,
                    msgs[:1]
                ).nanmean().item()
                metrics[f"bit_accuracy_{label.lower().replace(' ', '_').replace('.', '')}"] = bit_accuracy_
                metrics[f"pvalue_{label.lower().replace(' ', '_').replace('.', '')}"] = pvalue_
                metrics[f"capacity_{label.lower().replace(' ', '_').replace('.', '')}"] = capacity_
                # print(f"{label} - Bit Accuracy: {bit_accuracy_:.3f} - P-Value: {pvalue_:0.2e} - Capacity: {capacity_:.3f} - took {timer.stop():.2f}s")
        print(metrics)

        del vid, outputs, imgs, imgs_w, diff

Model loaded successfully from checkpoints/y_128b_img.pth with message: <All keys matched successfully>


Processing Videos for y_128b_img:   0%|          | 0/4 [00:00<?, ?it/s]

loading video assets/videos/metamoviegen/0.mp4 - took 7.56s
embedding watermark  - took 5.58s
compressing and detecting watermarks
{'psnr': 46.193031311035156, 'bit_accuracy_original': 1.0, 'pvalue_original': 2.938735877055719e-39, 'capacity_original': 128.0, 'bit_accuracy_h264_30_+_crop_08': 1.0, 'pvalue_h264_30_+_crop_08': 2.938735877055719e-39, 'capacity_h264_30_+_crop_08': 128.0, 'bit_accuracy_h264_40': 1.0, 'pvalue_h264_40': 2.938735877055719e-39, 'capacity_h264_40': 128.0, 'bit_accuracy_h264_50': 0.5859375, 'pvalue_h264_50': 0.03150439130416696, 'capacity_h264_50': 2.741180419921875}


Processing Videos for y_128b_img:  25%|██▌       | 1/4 [01:54<05:44, 114.78s/it]

loading video assets/videos/metamoviegen/1.mp4 - took 10.66s
embedding watermark  - took 6.42s
compressing and detecting watermarks
{'psnr': 44.6258544921875, 'bit_accuracy_original': 1.0, 'pvalue_original': 2.938735877055719e-39, 'capacity_original': 128.0, 'bit_accuracy_h264_30_+_crop_08': 1.0, 'pvalue_h264_30_+_crop_08': 2.938735877055719e-39, 'capacity_h264_30_+_crop_08': 128.0, 'bit_accuracy_h264_40': 1.0, 'pvalue_h264_40': 2.938735877055719e-39, 'capacity_h264_40': 128.0, 'bit_accuracy_h264_50': 0.5859375, 'pvalue_h264_50': 0.03150439130416696, 'capacity_h264_50': 2.741180419921875}


Processing Videos for y_128b_img:  50%|█████     | 2/4 [04:05<04:08, 124.03s/it]

loading video assets/videos/metamoviegen/2.mp4 - took 9.89s
embedding watermark  - took 6.63s
compressing and detecting watermarks
{'psnr': 45.25370407104492, 'bit_accuracy_original': 1.0, 'pvalue_original': 2.938735877055719e-39, 'capacity_original': 128.0, 'bit_accuracy_h264_30_+_crop_08': 1.0, 'pvalue_h264_30_+_crop_08': 2.938735877055719e-39, 'capacity_h264_30_+_crop_08': 128.0, 'bit_accuracy_h264_40': 0.9921875, 'pvalue_h264_40': 3.790969281401877e-37, 'capacity_h264_40': 119.56295776367188, 'bit_accuracy_h264_50': 0.765625, 'pvalue_h264_50': 6.452936410277732e-10, 'capacity_h264_50': 27.448287963867188}


Processing Videos for y_128b_img:  75%|███████▌  | 3/4 [06:12<02:05, 125.71s/it]

loading video assets/videos/metamoviegen/3.mp4 - took 9.80s
embedding watermark  - took 6.63s
compressing and detecting watermarks
{'psnr': 46.6988639831543, 'bit_accuracy_original': 1.0, 'pvalue_original': 2.938735877055719e-39, 'capacity_original': 128.0, 'bit_accuracy_h264_30_+_crop_08': 0.9765625, 'pvalue_h264_30_+_crop_08': 1.0274790409026221e-33, 'capacity_h264_30_+_crop_08': 107.47792053222656, 'bit_accuracy_h264_40': 1.0, 'pvalue_h264_40': 2.938735877055719e-39, 'capacity_h264_40': 128.0, 'bit_accuracy_h264_50': 0.7109375, 'pvalue_h264_50': 1.0030649725150208e-06, 'capacity_h264_50': 16.959083557128906}


Processing Videos for y_128b_img: 100%|██████████| 4/4 [08:19<00:00, 124.92s/it]
